In [2]:
#Imports and Dataset Paths
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm


In [4]:
# Update the base directory to match your structure
image_dir = r'C:/Users/sreevishak/Desktop/DUK/mini/datas/frames'

# Load the list of images for violence and non-violence
non_violence_images = os.listdir(os.path.join(image_dir, 'frames2nv'))
violence_images = os.listdir(os.path.join(image_dir, 'frames1'))

print("--------------------------------------\n")
print('The length of Non Violence images is', len(non_violence_images))
print('The length of Violence images is', len(violence_images))
print("--------------------------------------\n")

--------------------------------------

The length of Non Violence images is 100473
The length of Violence images is 117747
--------------------------------------



In [10]:
dataset = []
label = []
img_siz = (64, 64)

# Process non-violence images
for i, image_name in tqdm(enumerate(non_violence_images[:10000]), desc="Non Violence"):  # Process a subset (e.g., first 10,000 images)
    if image_name.split('.')[-1].lower() == 'jpg':  # Ensure the file extension matches
        image_path = os.path.join(image_dir, 'frames2nv', image_name)
        image = cv2.imread(image_path)
        if image is not None:  # Skip unreadable images
            image = Image.fromarray(image, 'RGB')
            image = image.resize(img_siz)  # Resize to reduce memory usage
            dataset.append(np.array(image))
            label.append(0)

# Process violence images
for i, image_name in tqdm(enumerate(violence_images[:10000]), desc="Violence"):  # Process a subset (e.g., first 10,000 images)
    if image_name.split('.')[-1].lower() == 'jpg':  # Ensure the file extension matches
        image_path = os.path.join(image_dir, 'frames1', image_name)
        image = cv2.imread(image_path)
        if image is not None:  # Skip unreadable images
            image = Image.fromarray(image, 'RGB')
            image = image.resize(img_siz)  # Resize to reduce memory usage
            dataset.append(np.array(image))
            label.append(1)

# Convert lists to numpy arrays
dataset = np.array(dataset, dtype=np.float32)  # Use float32 for normalization later
label = np.array(label, dtype=np.int32)

print("--------------------------------------\n")
print('Dataset Length: ', len(dataset))
print('Label Length: ', len(label))
print("--------------------------------------\n")

print("--------------------------------------\n")
print("Train-Test Split")
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=42)
print("--------------------------------------\n")

print("--------------------------------------\n")
print("Normalizing the Dataset.\n")

# Normalize the data
x_train = x_train / 255.0  # Normalize pixel values to [0, 1]
x_test = x_test / 255.0  # Normalize pixel value to [0, 1]

print("--------------------------------------\n")


Non Violence: 10000it [00:38, 257.50it/s]
Violence: 10000it [01:07, 148.36it/s]


--------------------------------------

Dataset Length:  20000
Label Length:  20000
--------------------------------------

--------------------------------------

Train-Test Split
--------------------------------------

--------------------------------------

Normalizing the Dataset.

--------------------------------------



In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

# Example: Load your image data into x_train and y_train
# Ensure x_train and y_train are loaded with actual data
# x_train, y_train = load_your_data()  # Replace with actual data loading code

# For example, if you're working with NumPy arrays:
# x_train = np.random.rand(1000, 64, 64, 3)  # Replace with your actual image data
# y_train = np.random.randint(0, 2, size=(1000,))  # Replace with actual labels (binary)

# Manually split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(1, activation='sigmoid')  # Binary classification (Violence vs Non-Violence)
])

# Print model summary
print("--------------------------------------\n")
print("Model Summary:\n")
model.summary()
print("--------------------------------------\n")     

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Adaptive learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Data Augmentation setup
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply data augmentation to training and validation data
train_datagen = datagen.flow(x_train, y_train, batch_size=32)
val_datagen = datagen.flow(x_val, y_val, batch_size=32)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model checkpoint to save the best model
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Train the model with data augmentation
print("--------------------------------------\n")
print("Training Started.\n")

history = model.fit(
    train_datagen,  # Use augmented training data
    epochs=20,  # Increase epochs for better training
    validation_data=val_datagen,  # Use augmented validation data
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

print("Training Finished.\n")
print("--------------------------------------\n")


--------------------------------------

Model Summary:



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,329 (2.61 MB)

 Trainable params: 683,329 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

--------------------------------------

--------------------------------------

Training Started.

Epoch 1/20


C:\Users\sreevishak\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


450/450 ━━━━━━━━━━━━━━━━━━━━ 31s 64ms/step - accuracy: 0.7670 - loss: 1.8955 - val_accuracy: 0.9237 - val_loss: 0.4918
Epoch 2/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 31s 68ms/step - accuracy: 0.9342 - loss: 0.4101 - val_accuracy: 0.9444 - val_loss: 0.2521
Epoch 3/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 28s 62ms/step - accuracy: 0.9492 - loss: 0.2296 - val_accuracy: 0.9681 - val_loss: 0.1644
Epoch 4/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 27s 59ms/step - accuracy: 0.9581 - loss: 0.1630 - val_accuracy: 0.9456 - val_loss: 0.1607
Epoch 5/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9650 - loss: 0.1326 - val_accuracy: 0.9787 - val_loss: 0.1054
Epoch 6/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 27s 60ms/step - accuracy: 0.9709 - loss: 0.1093 - val_accuracy: 0.9737 - val_loss: 0.0967
Epoch 7/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.9714 - loss: 0.1046 - val_accuracy: 0.9650 - val_loss: 0.1078
Epoch 8/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 27s 60ms/step - accuracy: 0.9745 - loss: 0.0947 - val_accurac

In [29]:
print("--------------------------------------\n")
print("Model Evaluation Phase.\n")

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

print("--------------------------------------\n")


--------------------------------------

Model Evaluation Phase.

125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9765 - loss: 0.0827
Test Loss: 0.0822
Test Accuracy: 97.55%
--------------------------------------



In [28]:
# Save the trained model
model.save('violence_detection_model.h5')
print("Model saved successfully.")


Model saved successfully.
